In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

from lol_win_predictor import (
    load_and_preprocess_data,
    aggregate_players_by_team,
    identify_key_features,
    train_win_rate_model,
    train_player_to_team_features,
    predict_win_rate_from_players,
    evaluate_team_composition,
    print_team_analysis,
    generate_random_player_stats,
    create_team_variants,
    create_optimal_team,
    save_models,
    load_models,
    plot_team_comparison,
    plot_role_impact,
    plot_optimization_progress
)

2025-03-21 22:06:18.843864: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-21 22:06:18.853740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742619978.863137  398941 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742619978.866161  398941 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-21 22:06:18.876495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Define folder paths
models_folder = 'models'
data_folder = 'data'

# Create models folder if it doesn't exist
if not os.path.exists(models_folder):
    os.makedirs(models_folder)
    print(f"Created folder: {models_folder}")

model_file_path = os.path.join(models_folder, 'lol_models.pkl')

if os.path.exists(model_file_path):
    print(f"\nLoading existing models from {model_file_path}...")
    feature_to_win_rate_model, player_to_feature_models, key_features = load_models(models_folder)
    print("Models loaded successfully!")
else:
    print(f"\nNo existing models found at {model_file_path}. Training new models...")
    
    # Load and preprocess data
    df_players, df_teams = load_and_preprocess_data(data_folder)
    
    # Aggregate player statistics by team
    print("\nAggregating player statistics by team...")
    df_aggregated = aggregate_players_by_team(df_players)
    
    # Identify key features for predicting win rate
    print("\nIdentifying key team features for win rate prediction...")
    key_features = identify_key_features(df_teams, n_features=8)
    
    # Train model to predict win rate from team features
    print("\nTraining win rate prediction model using key team features...")
    feature_to_win_rate_model = train_win_rate_model(df_teams, key_features)
    
    # Train models to predict team features from player statistics
    print("\nTraining models to predict key team features from player statistics...")
    player_to_feature_models = train_player_to_team_features(df_aggregated, df_teams, key_features)
    
    # Save the trained models
    save_models(feature_to_win_rate_model, player_to_feature_models, key_features, models_folder)

print(f"\nKey features used for prediction: {key_features}")


Loading existing models from models/lol_models.pkl...


I0000 00:00:1742619981.047683  398941 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13499 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Models loaded from models/lol_models.pkl
Models loaded successfully!

Key features used for prediction: ['K:D', 'GPM', 'GDM', 'Kills / game', 'Towers killed', 'Towers lost', 'NASH%', 'DPM']


In [3]:
strong_team = {
    'TOP': {'KDA': 3.5, 'Avg kills': 2.8, 'Avg deaths': 1.8, 'Avg assists': 5.5, 
            'CSM': 8.5, 'GPM': 400, 'DMG%': 0.24, 'DPM': 480, 'Avg WPM': 0.65,
            'GD@15': 150, 'CSD@15': 12, 'XPD@15': 200},
    'JUNGLE': {'KDA': 4.0, 'Avg kills': 3.0, 'Avg deaths': 2.0, 'Avg assists': 9.0, 
               'CSM': 5.5, 'GPM': 380, 'DMG%': 0.19, 'DPM': 420, 'Avg WPM': 0.9,
               'GD@15': 180, 'CSD@15': 15, 'XPD@15': 220},
    'MID': {'KDA': 4.5, 'Avg kills': 4.2, 'Avg deaths': 1.6, 'Avg assists': 6.5, 
            'CSM': 9.0, 'GPM': 430, 'DMG%': 0.32, 'DPM': 550, 'Avg WPM': 0.6,
            'GD@15': 200, 'CSD@15': 18, 'XPD@15': 250},
    'ADC': {'KDA': 5.0, 'Avg kills': 4.8, 'Avg deaths': 1.3, 'Avg assists': 6.0, 
            'CSM': 9.5, 'GPM': 450, 'DMG%': 0.35, 'DPM': 600, 'Avg WPM': 0.5,
            'GD@15': 250, 'CSD@15': 22, 'XPD@15': 280},
    'SUPPORT': {'KDA': 4.5, 'Avg kills': 1.2, 'Avg deaths': 2.0, 'Avg assists': 14.0, 
                'CSM': 1.2, 'GPM': 280, 'DMG%': 0.12, 'DPM': 250, 'Avg WPM': 1.8,
                'GD@15': 50, 'CSD@15': 2, 'XPD@15': 30}
}

# Create a weaker team
weak_team = {
    'TOP': {'KDA': 2.0, 'Avg kills': 1.5, 'Avg deaths': 3.0, 'Avg assists': 3.5, 
            'CSM': 7.0, 'GPM': 340, 'DMG%': 0.18, 'DPM': 380, 'Avg WPM': 0.4,
            'GD@15': -150, 'CSD@15': -8, 'XPD@15': -120},
    'JUNGLE': {'KDA': 2.2, 'Avg kills': 1.5, 'Avg deaths': 3.2, 'Avg assists': 5.5, 
               'CSM': 4.2, 'GPM': 320, 'DMG%': 0.15, 'DPM': 350, 'Avg WPM': 0.5,
               'GD@15': -80, 'CSD@15': -6, 'XPD@15': -100},
    'MID': {'KDA': 2.5, 'Avg kills': 2.2, 'Avg deaths': 3.0, 'Avg assists': 4.0, 
            'CSM': 7.2, 'GPM': 360, 'DMG%': 0.22, 'DPM': 400, 'Avg WPM': 0.4,
            'GD@15': -100, 'CSD@15': -7, 'XPD@15': -150},
    'ADC': {'KDA': 2.8, 'Avg kills': 2.5, 'Avg deaths': 2.8, 'Avg assists': 3.8, 
            'CSM': 7.8, 'GPM': 370, 'DMG%': 0.24, 'DPM': 420, 'Avg WPM': 0.3,
            'GD@15': -50, 'CSD@15': -4, 'XPD@15': -80},
    'SUPPORT': {'KDA': 2.2, 'Avg kills': 0.6, 'Avg deaths': 3.8, 'Avg assists': 7.5, 
                'CSM': 0.8, 'GPM': 220, 'DMG%': 0.07, 'DPM': 180, 'Avg WPM': 1.1,
                'GD@15': -30, 'CSD@15': -2, 'XPD@15': -50}
}

# Generate a random team
random_team = generate_random_player_stats()

In [4]:
print("\nAnalyzing strong team...")
strong_win_rate, strong_analysis = evaluate_team_composition(
    strong_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(strong_analysis)


Analyzing strong team...

----- Team Composition Analysis -----
Predicted Win Rate: 66.58%

Predicted Team Stats:
- K:D: 1.5775
- GPM: 1940.3921
- GDM: 249.7160
- Kills / game: 14.6191
- Towers killed: 9.0807
- Towers lost: 2.7715
- NASH%: 80.7685
- DPM: 2286.5295

Role Impact Analysis:
- TOP: Impact Score = 8.69
- JUNGLE: Impact Score = 7.41
- MID: Impact Score = 11.52
- ADC: Impact Score = 12.65
- SUPPORT: Impact Score = 5.43

Team Strengths:
- K:D: 1.58
- GPM: 1940.39
- GDM: 249.72
- Kills / game: 14.62
- DPM: 2286.53

No significant weaknesses detected.

Recommendations:
- This team composition has good potential for success.
- Maintain consistent performance to achieve the predicted win rate.


In [5]:
print("\nAnalyzing weak team...")
weak_win_rate, weak_analysis = evaluate_team_composition(
    weak_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(weak_analysis)


Analyzing weak team...

----- Team Composition Analysis -----
Predicted Win Rate: 26.13%

Predicted Team Stats:
- K:D: 0.4378
- GPM: 1628.1525
- GDM: -239.5022
- Kills / game: 7.8652
- Towers killed: 3.7935
- Towers lost: 9.2294
- NASH%: 37.0381
- DPM: 1749.4049

Role Impact Analysis:
- TOP: Impact Score = 6.11
- JUNGLE: Impact Score = 5.33
- MID: Impact Score = 7.54
- ADC: Impact Score = 8.29
- SUPPORT: Impact Score = 2.96

No significant strengths detected.

Team Weaknesses:
- K:D: 0.44
- GPM: 1628.15
- GDM: -239.50
- Kills / game: 7.87
- DPM: 1749.40

Recommendations:
- This team composition has a low predicted win rate.
- Consider adjusting player roles or selecting players with better synergy.
- The TOP player should focus on lane dominance and early-game CSing.
- The JUNGLE player should improve vision control for better map awareness.
- The MID player should aim for more damage output in team fights.
- The ADC player should focus on positioning to improve survival in fights.
- 

In [6]:
print("\nAnalyzing random team...")
random_win_rate, random_analysis = evaluate_team_composition(
    random_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print_team_analysis(random_analysis)


Analyzing random team...

----- Team Composition Analysis -----
Predicted Win Rate: 55.81%

Predicted Team Stats:
- K:D: 1.0751
- GPM: 1797.9748
- GDM: 18.9882
- Kills / game: 12.3630
- Towers killed: 6.8757
- Towers lost: 5.9850
- NASH%: 63.0247
- DPM: 2226.6346

Role Impact Analysis:
- TOP: Impact Score = 5.87
- JUNGLE: Impact Score = 5.98
- MID: Impact Score = 10.78
- ADC: Impact Score = 8.80
- SUPPORT: Impact Score = 4.60

Team Strengths:
- Kills / game: 12.36
- DPM: 2226.63

No significant weaknesses detected.

Recommendations:
- This team composition has good potential for success.
- Maintain consistent performance to achieve the predicted win rate.
- The ADC player should focus on positioning to improve survival in fights.


In [7]:
# Team Optimization Demo
print("Starting optimization of random team...")
print(f"Initial win rate: {random_win_rate:.4f} ({random_win_rate*100:.2f}%)")

# Run optimization
optimized_team, improvement_log = create_optimal_team(
    random_team, 
    player_to_feature_models, 
    feature_to_win_rate_model, 
    key_features,
    n_iterations=50  # Increase for better results
)

# Plot optimization progress
fig = plot_optimization_progress(improvement_log, title="Team Optimization Progress")
plt.savefig('optimization_progress.png')
plt.close()
print("Optimization progress plot saved to 'optimization_progress.png'")

# Analyze optimized team
optimized_win_rate, optimized_analysis = evaluate_team_composition(
    optimized_team, player_to_feature_models, feature_to_win_rate_model, key_features
)
print("\nAnalyzing optimized team...")
print_team_analysis(optimized_analysis)

# Compare original and optimized teams
print("\nImprovement Summary:")
print(f"Original win rate: {random_analysis['win_rate_percent']:.2f}%")
print(f"Optimized win rate: {optimized_analysis['win_rate_percent']:.2f}%")
print(f"Absolute improvement: {optimized_analysis['win_rate_percent'] - random_analysis['win_rate_percent']:.2f}%")
print(f"Relative improvement: {((optimized_analysis['win_rate'] / random_analysis['win_rate']) - 1) * 100:.2f}%")

Starting optimization of random team...
Initial win rate: 0.5581 (55.81%)
Iteration 10: Improved win rate to 0.6580
Optimization progress plot saved to 'optimization_progress.png'

Analyzing optimized team...

----- Team Composition Analysis -----
Predicted Win Rate: 66.05%

Predicted Team Stats:
- K:D: 1.2466
- GPM: 1884.5504
- GDM: 156.5889
- Kills / game: 13.9372
- Towers killed: 8.1908
- Towers lost: 4.4869
- NASH%: 77.3314
- DPM: 2354.8950

Role Impact Analysis:
- TOP: Impact Score = 6.07
- JUNGLE: Impact Score = 6.58
- MID: Impact Score = 9.66
- ADC: Impact Score = 8.71
- SUPPORT: Impact Score = 4.59

Team Strengths:
- K:D: 1.25
- GPM: 1884.55
- GDM: 156.59
- Kills / game: 13.94
- DPM: 2354.89

No significant weaknesses detected.

Recommendations:
- This team composition has good potential for success.
- Maintain consistent performance to achieve the predicted win rate.
- The ADC player should focus on positioning to improve survival in fights.

Improvement Summary:
Original win 

In [8]:
# Team Variant Comparison
# Create variants of the strong team
print("\nCreating variants of the strong team...")
team_variants = create_team_variants(strong_team, n_variants=3)

# Analyze all variants
variant_analyses = []
for i, variant in enumerate(team_variants):
    print(f"Analyzing Team Variant {i+1}...")
    _, analysis = evaluate_team_composition(
        variant, player_to_feature_models, feature_to_win_rate_model, key_features
    )
    variant_analyses.append(analysis)
    print(f"Win Rate: {analysis['win_rate_percent']:.2f}%")

# Plot comparison
fig = plot_team_comparison(variant_analyses, title="Team Variants Win Rate Comparison")
plt.savefig('team_variants_comparison.png')
plt.close()
print("Team variants comparison plot saved to 'team_variants_comparison.png'")


Creating variants of the strong team...
Analyzing Team Variant 1...
Win Rate: 66.58%
Analyzing Team Variant 2...
Win Rate: 66.58%
Analyzing Team Variant 3...
Win Rate: 66.58%
Analyzing Team Variant 4...
Win Rate: 66.58%
Team variants comparison plot saved to 'team_variants_comparison.png'
